In [9]:
import tensorflow as tf
import numpy as np
import pandas as pd
import preprocess_dataset as ut
from scipy.interpolate import BSpline
from tfkan import DenseKAN
from tensorflow import keras
from keras_tuner import HyperModel, GridSearch, RandomSearch


In [10]:
x_train = tf.convert_to_tensor(pd.read_csv("datasets/x_train.csv").to_numpy()[:, 1:])
y_train = tf.convert_to_tensor(pd.read_csv("datasets/y_train.csv").to_numpy()[:, 1:])

x_val = tf.convert_to_tensor(pd.read_csv("datasets/x_val.csv").to_numpy()[:, 1:])
y_val = tf.convert_to_tensor(pd.read_csv("datasets/y_val.csv").to_numpy()[:, 1:])

x_test = tf.convert_to_tensor(pd.read_csv("datasets/x_test.csv").to_numpy()[:, 1:])
y_test = tf.convert_to_tensor(pd.read_csv("datasets/y_test.csv").to_numpy()[:, 1:])

print(x_train.shape, x_val.shape, x_test.shape, y_train.shape, y_test.shape, y_val.shape)

(60000, 26) (15000, 26) (20000, 26) (60000, 1) (20000, 1) (15000, 1)


In [3]:
kan_layers = [4,1]
kan_model = ut.build_model(kan_layers, x_train.shape[1], mlp=False)
kan_model.summary()

mlp_layers = [20,3,1]
mlp_model = ut.build_model(mlp_layers, x_train.shape[1], mlp=True)
mlp_model.summary()
kan_history = kan_model.fit(x_train, y_train, epochs=20, batch_size=128, validation_data=(x_val, y_val), verbose=1)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_kan (DenseKAN)            │ (None, 4)              │           940 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_kan_1 (DenseKAN)          │ (None, 1)              │            37 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 977 (7.63 KB)

 Trainable params: 977 (7.63 KB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 20)             │           540 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │            63 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 607 (2.37 KB)

 Trainable params: 607 (2.37 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
547/547 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 442.7349 - mse: 442.7349 - val_loss: 25.7522 - val_mse: 25.7522
Epoch 2/20
547/547 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 17.8267 - mse: 17.8267 - val_loss: 8.8587 - val_mse: 8.8587
Epoch 3/20
547/547 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 7.6650 - mse: 7.6650 - val_loss: 5.6406 - val_mse: 5.6406
Epoch 4/20
547/547 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 5.1133 - mse: 5.1133 - val_loss: 4.1034 - val_mse: 4.1034
Epoch 5/20
547/547 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 3.7855 - mse: 3.7855 - val_loss: 2.9408 - val_mse: 2.9408
Epoch 6/20
547/547 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 2.7041 - mse: 2.7041 - val_loss: 2.0710 - val_mse: 2.0710
Epoch 7/20
547/547 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 1.8782 - mse: 1.8782 - val_loss: 1.4288 - val_mse: 1.4288
Epoch 8/20
547/547 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 1.3305 - mse: 1.3305 - val_loss: 1.1974 - val_mse: 1.1974
Epoch 9/20
547/547 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/s

In [11]:
# MLP Pura

pure_mlp_layers = [64, 32, 16, 1]
mlp_model = ut.build_model(pure_mlp_layers, x_train.shape[1], mlp=True, optimizer='rmsprop', loss="categorical_crossentropy")
mlp_model.summary()

mlp_history = mlp_model.fit(x_train, y_train, epochs=50, batch_size=128, validation_data=(x_test, y_test), verbose=1)

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_19 (Dense)                │ (None, 64)             │         1,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,353 (17.00 KB)

 Trainable params: 4,353 (17.00 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 4.7835e-07 - mse: 40.4156 - val_loss: 4.7640e-07 - val_mse: 40.5267
Epoch 2/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 4.7804e-07 - mse: 40.2649 - val_loss: 4.7640e-07 - val_mse: 40.5267
Epoch 3/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 4.7476e-07 - mse: 40.4069 - val_loss: 4.7640e-07 - val_mse: 40.5267
Epoch 4/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 4.7626e-07 - mse: 40.0418 - val_loss: 4.7640e-07 - val_mse: 40.5267
Epoch 5/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 4.7773e-07 - mse: 40.3373 - val_loss: 4.7640e-07 - val_mse: 40.5267
Epoch 6/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 4.7521e-07 - mse: 40.1416 - val_loss: 4.7640e-07 - val_mse: 40.5267
Epoch 7/50
234/547 ━━━━━━━━━━━━━━━━━━━━ 0s 859us/step - loss: 4.7539e-07 - mse: 39.8079

In [11]:
import tensorflow as tf
from tensorflow import keras
from kerastuner import HyperModel
from kerastuner.tuners import BayesianOptimization
from sklearn.model_selection import KFold

# Assumo che DenseKAN sia definito altrove nel tuo codice

class MyHyperModel(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape

    def build(self, hp):
        model = keras.Sequential()
        model.add(tf.keras.layers.Input(shape=self.input_shape))
        
        for i in range(hp.Int('num_layers', 2, 5)):
            model.add(DenseKAN(
                units=hp.Int('units_' + str(i), min_value=8, max_value=128, step=8),
                grid_size=hp.Int('grid_size_' + str(i), 3, 7),
                spline_order=hp.Int('spline_order_' + str(i), 2, 4)
            ))
        
        model.add(DenseKAN(1))

        learning_rate = hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')
        optimizer = hp.Choice('optimizer', ['adam', 'adagrad', 'rmsprop', 'adadelta'])
        
        if optimizer == 'adam':
            opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        elif optimizer == 'adagrad':
            opt = tf.keras.optimizers.Adagrad(learning_rate=learning_rate)
        elif optimizer == 'rmsprop':
            opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
        else:
            opt = tf.keras.optimizers.Adadelta(learning_rate=learning_rate)

        model.compile(optimizer=opt,
                      loss='mean_absolute_error',
                      metrics=[
                          tf.keras.metrics.MeanSquaredError(name='mse'),
                          tf.keras.metrics.MeanAbsoluteError(name='mae'),
                          tf.keras.metrics.RootMeanSquaredError(name='rmse')
                      ])
        return model

# Assumo che x_train, y_train siano già definiti
hypermodel = MyHyperModel(input_shape=(26,))

tuner = BayesianOptimization(
    hypermodel,
    objective='val_loss',
    max_trials=50,
    directory=r".\modelli_salvati",
    project_name='retina_kan'
)

tuner.search_space_summary()

# Callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6
)

# K-Fold Cross Validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_indices, val_indices) in enumerate(kfold.split(x_train)):
    print(f"Fold {fold + 1}")
    x_train_fold, x_val_fold = x_train[train_indices], x_train[val_indices]
    y_train_fold, y_val_fold = y_train[train_indices], y_train[val_indices]

    tuner.search(x_train_fold, y_train_fold,
                 epochs=100,
                 validation_data=(x_val_fold, y_val_fold),
                 callbacks=[early_stopping, lr_scheduler])

tuner.results_summary()
best_model = tuner.get_best_models(num_models=1)[0]

# Valutazione del modello migliore sul set di test
# Assumo che x_test, y_test siano già definiti
test_loss, test_mse, test_mae, test_rmse = best_model.evaluate(x_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test MSE: {test_mse}")
print(f"Test MAE: {test_mae}")
print(f"Test RMSE: {test_rmse}")

# Salvataggio del modello migliore
best_model.save('best_retina_kan_model.h5')

Search space summary
Default search space size: 9
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 5, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 8, 'max_value': 128, 'step': 8, 'sampling': 'linear'}
grid_size_0 (Int)
{'default': None, 'conditions': [], 'min_value': 3, 'max_value': 7, 'step': 1, 'sampling': 'linear'}
spline_order_0 (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 4, 'step': 1, 'sampling': 'linear'}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 8, 'max_value': 128, 'step': 8, 'sampling': 'linear'}
grid_size_1 (Int)
{'default': None, 'conditions': [], 'min_value': 3, 'max_value': 7, 'step': 1, 'sampling': 'linear'}
spline_order_1 (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 4, 'step': 1, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 's

TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got array([    0,     1,     2, ..., 59996, 59997, 59998])